In [2]:
#-----------------------------------------------------------------------------------------------------
# Connection to pyVMMonitor for profiling
#-----------------------------------------------------------------------------------------------------
import sys
sys.path.append('D:\\Program Files\\Brainwy\\PyVmMonitor 1.1.2\\public_api')
import pyvmmonitor
pyvmmonitor.connect()

In [3]:
# Libraries Imports
#-----------------------------------------------------------------------------------------------------
from configparser import ConfigParser
from sqlalchemy import create_engine
from datetime import datetime
import pandas as pd
import numpy as np
from datetime import date, timedelta

# Load connexion configuration file
#-----------------------------------------------------------------------------------------------------
config_filename = "./conf/GetWeather.ini"

# Read config.ini file
config_object = ConfigParser()
config_object.read(config_filename)
if  not config_object:
    print("Error while loading configuration !!!")

# Log initialization
cfg = config_object['INFO']
# Establish connection with DB (using sqlalchemy)
#-----------------------------------------------------------------------------------------------------
db_connection_str = "mysql+pymysql://{}:{}@{}/WeatherDB".format(cfg['user'],cfg['password'],cfg['host'])
db_connection = create_engine(db_connection_str)

In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : List records created by locations and creation dates
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
"""SELECT date(date_timestamp) AS dte, Locations.name, count(date_timestamp) AS nb_records
    FROM RawRecords
    INNER JOIN Locations ON Locations.id = RawRecords.id_location
    WHERE id_location IN (SELECT id FROM Locations)
    GROUP BY id_location, date(date_timestamp) 
    ORDER BY date(date_timestamp);""",
    con=db_connection
)
pdata = pd.DataFrame(raw_data)
pdata.head(100)

In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : List records created by locations and creation dates grouped 
#         by locations + month
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
""" SELECT date(date_timestamp) AS dte, Locations.name AS LName, count(date_timestamp) AS nb_records
    FROM RawRecords
    INNER JOIN Locations ON Locations.id = RawRecords.id_location
    WHERE MONTH(DATE(date_timestamp)) in (SELECT MONTH(DATE(date_timestamp)) FROM RawRecords GROUP BY MONTH(DATE(date_timestamp)))
    AND ( id_location IN (SELECT id FROM Locations) ) 
    GROUP BY id_location, month(date_timestamp) 
    ORDER BY id_location, month(date_timestamp);""",
    con=db_connection  
)
pdata = pd.DataFrame(raw_data)
pdata.head(100)

In [5]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : Count all records
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
""" SELECT COUNT(*) AS totalRecs 
    FROM RawRecords;""",
    con=db_connection
)
pdata = pd.DataFrame(raw_data)
pdata.head(10)

,totalRecs
0,92507


In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : Count sorted number of records per locations
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
    """ SELECT Locations.name as name, COUNT(*) as nbRecords
        FROM RawRecords
        INNER JOIN Locations ON RawRecords.id_location = Locations.id
        GROUP BY id_location
        ORDER BY nbRecords DESC;""", 
        con=db_connection
)
pdata = pd.DataFrame(raw_data)
pdata.head(10)

In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# List records created by locations and creation dates
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
""" SELECT date(date_timestamp) AS dte, Locations.name, count(date_timestamp) AS nb_records
    FROM RawRecords
    INNER JOIN Locations ON Locations.id = RawRecords.id_location
    WHERE id_location IN (SELECT id FROM Locations)
    GROUP BY id_location, date(date_timestamp) 
    ORDER BY date(date_timestamp);""",
    con=db_connection
)
pdata = pd.DataFrame(raw_data)
pdata.tail(100)

In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : List records created by locations and creation dates grouped 0
#         by locations + month
#----------------------------------------------------------------------------------

raw_data = pd.read_sql(
""" SELECT date(date_timestamp) AS dte, Locations.name AS LName, count(date_timestamp) AS nb_records
    FROM RawRecords
    INNER JOIN Locations ON Locations.id = RawRecords.id_location
    WHERE MONTH(DATE(date_timestamp)) in (SELECT MONTH(DATE(date_timestamp)) FROM RawRecords GROUP BY MONTH(DATE(date_timestamp)))
    AND ( id_location IN (SELECT id FROM Locations) ) 
    GROUP BY id_location, month(date_timestamp) 
    ORDER BY id_location, month(date_timestamp);""",
    con=db_connection
)
pdata = pd.DataFrame(raw_data)
pdata.tail(100)

In [7]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Select MIN,MAX,AVG by locations and creation dates grouped by locations + day
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
""" SELECT date(date_timestamp) AS dte, Locations.name AS LName, 
    (round(min(temp),2) - 273.15) as minTemp, (round(max(temp),2) - 273.15) as maxTemp, (round(avg(temp), 2) - 273.15) as avgTemp,
    round(min(humidity),2) as minHumidity, round(max(humidity),2) as maxHumidity, round(avg(humidity), 2) as avgHumidity,
    round(min(pressure),2) as minPressure, round(max(pressure),2) as maxPressure, round(avg(pressure), 2) as avgPressure,
    round(sum(rain_1h))
    FROM RawRecords
    INNER JOIN Locations ON Locations.id = RawRecords.id_location
    WHERE id_location IN (SELECT id FROM Locations)
    GROUP BY id_location, month(date_timestamp),day(date_timestamp)
    ORDER BY id_location, date_timestamp; """,
    con=db_connection
)
pdata = pd.DataFrame(raw_data)
pdata.tail(100)

,dte,LName,minTemp,maxTemp,avgTemp,minHumidity,maxHumidity,avgHumidity,minPressure,maxPressure,avgPressure,round(sum(rain_1h))
923,2021-11-19,Hérémence,-2.16,11.00,2.31,48.0,100.0,77.84,1027.0,1033.0,1031.01,0.0
924,2021-11-20,Hérémence,-1.57,12.40,3.78,53.0,93.0,75.62,1020.0,1031.0,1025.41,0.0
925,2021-11-21,Hérémence,-3.16,10.03,3.00,47.0,100.0,72.91,1008.0,1020.0,1012.84,1.0
926,2021-11-22,Hérémence,0.97,4.62,2.14,67.0,88.0,81.40,1010.0,1018.0,1013.23,0.0
927,2021-11-23,Hérémence,-0.25,5.59,2.26,59.0,90.0,79.10,1018.0,1023.0,1020.78,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1018,2022-02-22,Hérémence,-1.20,5.01,1.84,40.0,91.0,68.12,1018.0,1028.0,1024.47,9.0
1019,2022-02-23,Hérémence,-3.39,8.74,2.13,33.0,87.0,63.62,1023.0,1028.0,1026.10,0.0
1020,2022-02-24,Hérémence,-2.23,7.50,2.30,28.0,93.0,56.72,1017.0,1025.0,1020.45,0.0
1021,2022-02-25,Hérémence,-3.49,4.80,1.11,20.0,93.0,48.18,1019.0,1037.0,1024.25,0.0


In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Select MIN,MAX,AVG by locations and creation dates grouped by locations + month
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
""" SELECT date(date_timestamp) AS dte, Locations.name AS LName, 
    (round(min(temp),2) - 273.15) as minTemp, (round(max(temp),2) - 273.15) as maxTemp, (round(avg(temp), 2) - 273.15) as avgTemp,
    round(min(humidity),2) as minHumidity, round(max(humidity),2) as maxHumidity, round(avg(humidity), 2) as avgHumidity,
    round(min(pressure),2) as minPressure, round(max(pressure),2) as maxPressure, round(avg(pressure), 2) as avgPressure
    FROM RawRecords
    INNER JOIN Locations ON Locations.id = RawRecords.id_location
    WHERE MONTH(DATE(date_timestamp)) in (SELECT MONTH(DATE(date_timestamp)) FROM RawRecords GROUP BY MONTH(DATE(date_timestamp)))
    AND ( id_location IN (SELECT id FROM Locations) ) 
    GROUP BY id_location, month(date_timestamp) 
    ORDER BY id_location, month(date_timestamp);""",
    con=db_connection
)
pdata = pd.DataFrame(raw_data)
pdata.head(100)

In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : Count frozen days this year by location(s)
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
    """
        SELECT LocationName, count(date)
        FROM (
                SELECT date(date_timestamp) AS date, Locations.name AS LocationName
                FROM RawRecords
                INNER JOIN Locations ON Locations.id = RawRecords.id_location
                WHERE (id_location ) 
                GROUP BY id_location, MONTH(date_timestamp), DAY(date_timestamp)
                HAVING ( (round(max(temp), 2) - 273.15) < 0)
        ) tmpTable
        GROUP BY tmpTable.LocationName
        ;
    """,
    con=db_connection
)

pdata = pd.DataFrame(raw_data)
pdata.head(100)


In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : Count cold days (below 10°C) this year by location(s)
#----------------------------------------------------------------------------------

raw_data =  pd.read_sql(
    """
        SELECT LocationName, count(date)
        FROM (
                SELECT date(date_timestamp) AS date, Locations.name AS LocationName
                FROM RawRecords
                INNER JOIN Locations ON Locations.id = RawRecords.id_location
                WHERE (id_location ) 
                GROUP BY id_location, MONTH(date_timestamp), DAY(date_timestamp)
                HAVING ( (round(max(temp), 2) - 273.15) < 10)
        ) tmpTable
        GROUP BY tmpTable.LocationName
        ;
    """,
    con=db_connection
)

pdata = pd.DataFrame(raw_data)
pdata.head(100)

In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : Count hot days (over 20°C including night) this year by location(s)
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
    """
        SELECT LocationName, count(date)
        FROM (
                SELECT date(date_timestamp) AS date, Locations.name AS LocationName
                FROM RawRecords
                INNER JOIN Locations ON Locations.id = RawRecords.id_location
                WHERE (id_location ) 
                GROUP BY id_location, MONTH(date_timestamp), DAY(date_timestamp)
                HAVING ( (round(min(temp), 2) - 273.15) >= 20)
        ) tmpTable
        GROUP BY tmpTable.LocationName
        ;
    """,
    con=db_connection
)

pdata = pd.DataFrame(raw_data)
pdata.head(100)

In [ ]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : Count snow days  this year by location(s) anmd date
#----------------------------------------------------------------------------------
raw_data = pd.read_sql(
    """
        SELECT LocationName, count(date) AS snowDays
        FROM (
                SELECT date(date_timestamp) AS date, Locations.name AS LocationName, round((temp - 273.15),2) AS t
                FROM RawRecords
                INNER JOIN Locations ON Locations.id = RawRecords.id_location
                WHERE (id_location AND rain_1h > 0 ) 
                GROUP BY id_location, YEAR(date_timestamp), MONTH(date_timestamp), DAY(date_timestamp)
                HAVING ( t <= 2)
        ) tmpTable
        GROUP BY tmpTable.LocationName
        ;
    """,
    con=db_connection
)

pdata = pd.DataFrame(raw_data)
pdata.head(100)

In [8]:
#----------------------------------------------------------------------------------
# Table : RawRecords
# Query : Count rain/snow consecutive days and return a list of periods
#----------------------------------------------------------------------------------
import datetime
raw_data = pd.read_sql(
    """
        SELECT date(date_timestamp) AS date, Locations.name AS LocationName, round(avg(rain_1h) * 24,2) AS rain
        FROM RawRecords
        INNER JOIN Locations ON Locations.id = RawRecords.id_location
        WHERE (id_location = 6) AND (rain_1h > 0 ) 
        GROUP BY id_location, MONTH(date_timestamp), DAY(date_timestamp)
        HAVING ( rain > 0)
    """,
    con=db_connection
)

pdata = pd.DataFrame(raw_data)
tmpList = []
patternList = []
lastDate = datetime.datetime(1970,1,1)

for item in pdata.iterrows():
        if (item[1]['date'] != (lastDate + datetime.timedelta(days=1))):
            patternList.append(tmpList)
            tmpList = []        
            tmpList.append(item[1]) 
        lastDate = item[1]['date']
